In [166]:
import numpy as np
import json
import os
import pandas as pd
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import re

In [167]:
#Toma los archivos .json del directorio y los une a una base de datos usando pandas.

data_list = []
base_dir = 'tech---ops_Actualizada/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df = pd.concat(data_list)


In [168]:
# Se agregaron las columnas nuevas segun las reacciones y ademas se ordenaron por el index.

nuevas_columnas_reacciones = ['t_finalizado', 't_revisado', 't_resuelto']

df_columnas_reaccion = df.reindex(columns=df.columns.tolist() + nuevas_columnas_reacciones)
df_columnas_reaccion['reactions'].fillna('', inplace=True)
df_columnas_reaccion = df_columnas_reaccion.reset_index(drop=True)



In [169]:
def merge_dict(lista):
    """Funci'on que une diccionarios en uno solo a travez de listas

    Args:
        lista (list): toma como argumdento una lista no vacia!!! que contenga varios diccionarios
    """
    new_dict = {'name':[], 'users':[]}

    for elemento in lista:
        new_dict['name'].append(elemento['name'])
        new_dict['users'].append(elemento['users'])

    return new_dict

In [170]:
# Se creo una lista vacia para almacenar todos los diccionarios de la columna reactions

lista_dict = []

# Se itero la columna reactions para almacenar los respectivos diccionarios usando la funcion anteriormente creada en lista_dict para despues generar un DataFrame con los datos

for i in df_columnas_reaccion.index:
    if df_columnas_reaccion['reactions'][i] != '':
        lista_dict.append(merge_dict(df_columnas_reaccion['reactions'][i]))
    else:
        lista_dict.append({'name': [], 'users': []})

df_reaction_provisoria = pd.DataFrame(lista_dict)

In [ ]:
# Se creo una lista con los simbolos de reaccion

simbolos_reaccion = ['white_check_mark', 'eyes', 'raised_hands']

# Se itero la base de datos con el fin de almacenar True's & False's en las respectivas columnas de su reaccion

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[0] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_finalizado'][i] = True

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[1] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_revisado'][i] = True

for i in df_reaction_provisoria.index:
    if any(simbolos_reaccion[2] in s for s in df_reaction_provisoria['name'][i]):
        df_columnas_reaccion['t_resuelto'][i] = True

df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']] = df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']].fillna(value=False)

In [172]:
# Seleccionamos las columnas utiles del df y ademas se elimnaron las filas duplicadas y se ordenaron segun la fecha y hora de la creacion de primer mensaje. 

df_clean = df_columnas_reaccion[['client_msg_id', 'text', 'user', 'ts', 'thread_ts', 'latest_reply', 't_finalizado', 't_revisado', 't_resuelto']].drop_duplicates().sort_values(['ts'])

df_clean = df_clean[df_clean['ts'] > '1642171431.027300'].reset_index(drop=True)

In [173]:
# Columnas nuevas a agregar

nuevas_columnas = ['nuevo_req', 'ticket_ops', 'ticket_tech', 'urgente']

df_clean_new_columns = df_clean.reindex(columns=df_clean.columns.tolist() + nuevas_columnas)
df_clean_new_columns['latest_reply'].fillna('', inplace=True)

In [ ]:
# Se creo una lista con los simbolos asociado a las nuevas columnas.

simbolos = [':writing_hand:', ':dart:', ':robot_face:',':rotating_light:']

# Se itero el df para asociarle un valor de True or False a las respectivas columnas segun la simbologia del mensaje principal del thread.

for i in df_clean_new_columns.index:
    if df_clean_new_columns['latest_reply'][i] != '':
            for j in range(len(simbolos)):
                if simbolos[j] in df_clean_new_columns['text'][i]:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = True
                else:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = False 

In [175]:
# Cambiar el nombre de la columna user a Owner

df_clean_new_columns = df_clean_new_columns.rename(columns = {'user': 'Owner'}, inplace=False)

In [ ]:
# Mostrar las primeras 25 filas del DataFrame

df_clean_new_columns.head(25)

In [ ]:
# Se creo una nueva DF para almacenar en una nueva columna el id de usuario del owner de cada ticket

df_owner = df_clean_new_columns.filter(items=['text', 'Owner', 'ts', 'thread_ts', 'latest_reply'])
df_owner = df_owner[df_owner['text'].str.contains('Owner', flags=re.IGNORECASE)]
columna_owner = df_owner['text']
df_owner['ticket_owner'] = columna_owner.apply(lambda st: st[st.find("@")+1:st.find(">")])

df_owner

In [ ]:
df_owner_transitoria = pd.merge(df_owner.filter(items=['thread_ts', 'ticket_owner']), df_clean_new_columns, on='thread_ts', how='outer')

df_owner_transitoria.head()

In [179]:
# Se creo el DataFrame para los users

data_list = []
base_dir = 'user-data/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df_users = pd.concat(data_list)
df_users = df_users[['id', 'name']]

In [181]:
# Cambiar el nombre de la columna id a ticket_owner

df_users = df_users.rename(columns = {'id': 'ticket_owner'}, inplace=False)

In [182]:
# Combinar el df de user con el df original para obtener los nombres de cada mensaje

data_clean1 = pd.merge(df_users, df_owner_transitoria, on='ticket_owner', how='outer')

data_clean1 = data_clean1[data_clean1['name'].notna()]
data_clean1 = data_clean1[data_clean1['latest_reply'] != ''].sort_values(['ts']).reset_index(drop=True)

In [183]:
# DataFrame transitoria para anadir la columna user_ticket que hace referencia al que genero el ticket

df_users_tickets = df_users[['ticket_owner', 'name']]
df_users_tickets = df_users_tickets.rename(columns = {'name': 'user_ticket', 'ticket_owner': 'Owner'}, inplace=False)

In [ ]:
# Combine el df df_user_tickets con data_clean1 para anadirle la columna user_ticket

data_clean1 = pd.merge(df_users_tickets, data_clean1, on='Owner', how='outer')
data_clean1.head()

In [185]:
# Eliminar columnas redundantes

data_clean1 = data_clean1.drop(['ticket_owner', 'client_msg_id', 'Owner'], axis=1)
data_clean1.head()

,user_ticket,name,thread_ts,text,ts,latest_reply,t_finalizado,t_revisado,t_resuelto,nuevo_req,ticket_ops,ticket_tech,urgente
0,jmansilla,juan,1642522495.055200,:writing_hand: :robot_face: Nuevo requerimient...,1642522495.055200,1643120976.006300,True,False,False,True,False,True,False
1,jmansilla,jmansilla,1643121323.008200,:writing_hand: :robot_face: GTD solicitó que s...,1643121323.008200,1643391317.053849,True,False,True,True,False,True,False
2,jmansilla,jmansilla,1643121350.008800,:dart: GTD reportó que no podía subir opcione...,1643121350.008800,1643391332.064719,True,False,True,False,True,False,False
3,jmansilla,nsanhueza,1643638020.980979,:rotating_light: :robot_face: Nuevo motor no ...,1643638020.980979,1643725766.005949,True,True,True,False,False,True,True
4,jmansilla,istuardo,1643743029.421769,:writing_hand: :robot_face: :dart: Necesitamo...,1643743029.421769,1643754322.519079,True,True,True,True,True,True,False


In [186]:
# Se tranformo de Unix TimeStamp a Pandas Datetime las columnas ts, thread_ts y latest_reply

data_clean1[['ts', 'thread_ts', 'latest_reply']] = data_clean1[['ts', 'thread_ts', 'latest_reply']].apply(pd.to_datetime, unit='s')

In [187]:
# Agregar nueva columna para encontrar el tiempo que estuvo abierto cada ticket.

data_clean1['Duracion_Ticket_horas'] = (data_clean1.latest_reply - data_clean1.ts) / pd.Timedelta(hours=1)

# Ordenar las columnas

data_clean1 = data_clean1[['user_ticket','name','text','ts','thread_ts','latest_reply','t_revisado','t_resuelto','t_finalizado','nuevo_req','ticket_ops','ticket_tech','urgente','Duracion_Ticket_horas']]
data_clean1 = data_clean1.rename(columns = {'name': 'ticket_owner'}, inplace=False)
data_clean1 = data_clean1[data_clean1['text'].notna()].sort_values(['ts']).reset_index(drop=True)

In [ ]:
# Descripcion analitica de la columna Duracion_tickets_horas para los tickets Resueltos

data_clean1[data_clean1['t_resuelto'] == True].describe()

In [189]:
# Funnel entre tickets revisados, resueltos y finalizados

fig = go.Figure(go.Funnel(
    y = data_clean1.columns[6:10] ,
    x = [data_clean1.t_revisado.value_counts().loc[True], data_clean1.t_resuelto.value_counts().loc[True], data_clean1.t_finalizado.value_counts().loc[True]]))

fig.show()

In [190]:
# Tarjeta que muestra la duracion promedio de los tickets resueltos recibidos en Slack

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = data_clean1[data_clean1['t_resuelto'] == True].describe()['Duracion_Ticket_horas']['mean'],
    number = {'suffix' : ' Horas'},
    title = {"text": "<span style='font-size:1.5em'>Duracion Promedio Tickets Resueltos</span><br><span style='font-size:1.5em'></span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

fig.show()

In [191]:
# Tiempo Duracion Ticket Urgente - Ops

tickets_ops = data_clean1[data_clean1['urgente'] == True]

tickets_ops_time = tickets_ops[tickets_ops['ticket_ops'] == True]

tickets_ops_time[tickets_ops_time['t_resuelto'] == True].describe()

,Duracion_Ticket_horas
count,10.000000
mean,134.173811
std,127.194951
min,1.059723
25%,34.548525
50%,95.831249
75%,211.347632
max,359.712521


In [192]:
# Tiempo Duracion Ticket Urgente - Tech

tickets_tech = data_clean1[data_clean1['urgente'] == True]

tickets_tech_time = tickets_tech[tickets_tech['ticket_tech'] == True]

tickets_tech_time[tickets_tech_time['t_resuelto'] == True].describe()

,Duracion_Ticket_horas
count,17.000000
mean,45.009779
std,82.324699
min,0.001944
25%,2.470557
50%,19.803328
75%,24.373618
max,313.569539


In [193]:
# Tiempo Duracion Ticket Nuevo Requerimiento - Ops

tickets_req_ops = data_clean1[data_clean1['nuevo_req'] == True]

tickets_req_ops_time = tickets_req_ops[tickets_req_ops['ticket_ops'] == True]

tickets_req_ops_time[tickets_req_ops_time['t_resuelto'] == True].describe()

,Duracion_Ticket_horas
count,8.000000
mean,88.131195
std,60.652101
min,3.136971
25%,56.558582
50%,69.340444
75%,125.118891
max,195.456309


In [194]:
# Tiempo Duracion Ticket Nuevo Requerimiento - Tech

tickets_req_tech = data_clean1[data_clean1['nuevo_req'] == True]

tickets_req_tech_time = tickets_req_tech[tickets_req_tech['ticket_tech'] == True]

tickets_req_tech_time[tickets_req_tech_time['t_resuelto'] == True].describe()


,Duracion_Ticket_horas
count,11.000000
mean,94.255688
std,53.713669
min,3.136971
25%,64.518446
50%,74.998346
75%,139.549146
max,167.075820


In [195]:
# Tarjeta que muestra la duracion promedio de los tickets de Nuevo Requerimiento recibidos por Operaciones

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = tickets_req_ops_time[tickets_req_ops_time['t_resuelto'] == True].describe()['Duracion_Ticket_horas']['mean'],
    number = {'suffix' : ' Horas'},
    title = {"text": "<span style='font-size:1.5em'>Duracion Promedio Tickets</span><br><span style='font-size:1.4em'>Nuevo Requerimiento Ops</span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

In [196]:
# Tarjeta que muestra la duracion promedio de los tickets urgente recibidos por Operaciones

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = tickets_ops_time[tickets_ops_time['t_resuelto'] == True].describe()['Duracion_Ticket_horas']['mean'],
    number = {'suffix' : ' Horas'},
    title = {"text": "<span style='font-size:1.5em'>Duracion Promedio Tickets</span><br><span style='font-size:1.4em'>Urgente Ops</span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

In [197]:
# Tarjeta que muestra la duracion promedio de los tickets de Nuevo Requerimiento recibidos por Tech

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = tickets_req_tech_time[tickets_req_tech_time['t_resuelto'] == True].describe()['Duracion_Ticket_horas']['mean'],
    number = {'suffix' : ' Horas'},
    title = {"text": "<span style='font-size:1.5em'>Duracion Promedio Tickets</span><br><span style='font-size:1.4em'>Nuevo Requerimiento Tech</span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

In [198]:
# Tarjeta que muestra la duracion promedio de los tickets Urgentes de Tech

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = tickets_tech_time[tickets_tech_time['t_resuelto'] == True].describe()['Duracion_Ticket_horas']['mean'],
    number = {'suffix' : ' Horas'},
    title = {"text": "<span style='font-size:1.5em'>Duracion Promedio Tickets</span><br><span style='font-size:1.4em'>Urgente Tech</span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

In [199]:
# Muestra la cantidad Total de Tickets Recibidos

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = data_clean1.describe()['Duracion_Ticket_horas']['count'],
    title = {"text": "<span style='font-size:1.5em'>Cantidad de Tickets</span><br><span style='font-size:1.5em'></span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

fig.show()

In [ ]:
# Se calcularon la cantidad de tickets Urgentes recibidos por Tech y Ops

valor_ops = data_clean1.ticket_ops.value_counts().loc[True]
df_suma_provisoria_ops = data_clean1[['ticket_ops', 'urgente']]
df_suma_provisoria_ops['suma'] = data_clean1[['ticket_ops', 'urgente']].sum(1)
valor_ops_urgente =df_suma_provisoria_ops['suma'].value_counts().loc[2.0]

valor_tech = data_clean1.ticket_tech.value_counts().loc[True]
df_suma_provisoria_tech = data_clean1[['ticket_tech', 'urgente']]
df_suma_provisoria_tech['suma'] = data_clean1[['ticket_tech', 'urgente']].sum(1)
valor_tech_urgente = df_suma_provisoria_tech['suma'].value_counts().loc[2.0]

In [201]:
# Se muestra en un grafico de barras agrupado en grupos, la cantidad de tickets urgentes recibidos por Operaciones y Tech

fig = go.Figure(data=[
    go.Bar(name='No Urgente', x=data_clean1.columns[10:12], y=[valor_ops - valor_ops_urgente, valor_tech - valor_tech_urgente]),
    go.Bar(name='Urgente', x=data_clean1.columns[10:12], y=[valor_ops_urgente, valor_tech_urgente])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [ ]:
# Se calcularon la cantidad de Nuevos Requerimientos recibidos por Tech y Operaciones

valor_req = data_clean1.nuevo_req.value_counts().loc[True]
df_suma_provisoria_req = data_clean1[['nuevo_req','ticket_ops']]
df_suma_provisoria_req['suma'] = data_clean1[['nuevo_req','ticket_ops']].sum(1)
valor_req_ops =df_suma_provisoria_req['suma'].value_counts().loc[2.0]

In [204]:
# Se muestra la cantidad de Nuevos Requerimientos para Tech y Ops

fig = go.Figure(data=[
    go.Bar(name='Nuevo Requerimiento Ops', x=data_clean1.columns[10:11], y=[valor_req_ops]),
    go.Bar(name='Nuevo Requerimiento Tech', x=data_clean1.columns[11:12], y=[valor_req - valor_req_ops])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [205]:
tickets_resueltos = data_clean1[data_clean1['t_resuelto'] == True]
df_count_owner_resueltos = tickets_resueltos['ticket_owner'].value_counts().to_frame().reset_index()
df_count_owner_resueltos = df_count_owner_resueltos.rename(columns= {'index': 'ticket_owner', 'ticket_owner': 'Count'}, inplace= False)

In [206]:
fig = px.bar(df_count_owner_resueltos, x='ticket_owner', y='Count', title='Tickets Resueltos')
fig.show()

In [207]:
tickets_abiertos = data_clean1[(data_clean1['t_revisado'] == True) & (data_clean1['t_resuelto'] == False) & (data_clean1['t_finalizado'] == False)]
df_count_owner_abiertos = tickets_abiertos['ticket_owner'].value_counts().to_frame().reset_index()
df_count_owner_abiertos = df_count_owner_abiertos.rename(columns= {'index': 'ticket_owner', 'ticket_owner': 'Count'}, inplace= False)

In [208]:
fig = px.bar(df_count_owner_abiertos, x='ticket_owner', y='Count', title='Tickets Abiertos')
fig.show()

In [209]:
# Tickets Asignados pero no revisados, resulto y finalizado

data_clean1[(data_clean1['t_revisado'] == False) & (data_clean1['t_resuelto'] == False) & (data_clean1['t_finalizado'] == False)]

,user_ticket,ticket_owner,text,ts,thread_ts,latest_reply,t_revisado,t_resuelto,t_finalizado,nuevo_req,ticket_ops,ticket_tech,urgente,Duracion_Ticket_horas
31,paula,jmansilla,:writing_hand: :robot_face: Generar reinicio ...,2022-01-24 14:11:39.005100012,2022-01-24 14:11:39.005100012,2022-02-02 14:27:31.922508955,False,False,False,True,False,True,False,216.264699
34,david,jmansilla,:rotating_light::dart: Realizar integraciones ...,2022-01-24 15:27:43.010900021,2022-01-24 15:27:43.010900021,2022-02-03 14:47:35.833138943,False,False,False,False,True,False,True,239.331340
60,NaN,jmansilla,This message was deleted.,2022-01-31 15:42:22.677758932,2022-01-31 15:42:22.677758932,2022-01-31 16:54:23.410109043,False,False,False,False,False,False,False,1.200203
65,vprieto,jmansilla,:writing_hand::skin-tone-3: :robot_face: Agreg...,2022-02-02 17:52:30.876029015,2022-02-02 17:52:30.876029015,2022-02-03 15:01:36.758378983,False,False,False,True,False,True,False,21.151634


In [210]:
# Exportando la data a excel en formato csv, ya que, en formato xlsx no permite exportarlo por el formato de las fechas

data_clean1.to_csv('data_slack_actualizada.csv')